# Reproducibility as a Service Data Analysis
This notebook analyzes the data collected by executing R scripts with and without the retroactive reproducibility tool RaaS.
All plots and tables displayed in the paper are generated here.

Imports
---------

In [1]:
import os
import sqlite3
import json
import requests
import re
import matplotlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from glob import glob

from helper_functions import *

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 25}
matplotlib.rc('font', **font)

%matplotlib inline

Analyzing scripts that ran without RaaS
=======================================

These scripts executed in a rocker/tidyverse environment, R version 3.6.3, with a time limit of one script per hour, up to five hours per dataset.

In [2]:
con = sqlite3.connect("../data/results.db")

scripts_df = pd.read_sql_query("SELECT * FROM results", con) 

In [3]:
scripts_df["doi"] = get_doi_from_results_filename_v(scripts_df["filename"])
scripts_df["error_category"] = determine_error_cause_v(scripts_df["error"])

Comparison of Chen's 2018 Study to our 2022 Study
------------------------------------------------

__Comparison of Total Errors and Successes__

In [4]:
total_num_scripts = len(scripts_df.index)
num_success_scripts = len(scripts_df[scripts_df["error_category"] == "success"].index)
num_error_scripts = len(scripts_df[scripts_df["error_category"] != "success"].index)

In [5]:
chen_comparison_markdown = '''
------------------------------------------------
              Chen's              Ours 
  --------- -------- --------- ------- ---------
  Result       Count   Percent   Count   Percent

  Success        408     14.4%   OUR_SUCCESS_COUNT     OUR_SUCCESS_PERCENT%

  Error         2431     85.6%   OUR_ERROR_COUNT    OUR_ERROR_PERCENT%

  Total         2839             OUR_TOTAL 
------------------------------------------------


Table: The occurrences of errors in scripts from Dataverse without
processing through a reproducibility framework. The first set of
results are from Chen's study in 2018, and the second is ours
conducted in 2022. The percents are rounded to the nearest tenth. {#tbl:error-occurrences}
'''

chen_comparison_markdown = chen_comparison_markdown.replace("OUR_SUCCESS_COUNT", str(num_success_scripts))
chen_comparison_markdown = chen_comparison_markdown.replace("OUR_SUCCESS_PERCENT", "{0:.4g}".format(num_success_scripts / total_num_scripts * 100))

chen_comparison_markdown = chen_comparison_markdown.replace("OUR_ERROR_COUNT", str(num_error_scripts))
chen_comparison_markdown = chen_comparison_markdown.replace("OUR_ERROR_PERCENT", "{0:.4g}".format(num_error_scripts / total_num_scripts * 100))

chen_comparison_markdown = chen_comparison_markdown.replace("OUR_TOTAL", str(total_num_scripts))

write_file_from_string("chen_total_comparison.md", chen_comparison_markdown)

__Comparison of Error Categories__

In [6]:
category_comparison_md = '''
------------------------------------------------------------
                        containR              RaaS 
  ------------------- ---------- --------- ------- ---------
  Error Type               Count   Percent   Count   Percent

  Library                    363     14.9%    LIBRARY_COUNT    LIBRARY_PERCENT%

  Working directory          696     28.6%    WD_COUNT    WD_PERCENT%

  Missing file               802     33.0%    FILE_COUNT    FILE_PERCENT%

  Function                    NA        NA     FUNC_COUNT     FUNC_PERCENT%

  Other                      569     23.4%    OTHER_COUNT    OTHER_PERCENT%

  Total                     2431              ERROR_TOTAL 

------------------------------------------------------------

Table: The most common causes of errors in scripts from Dataverse without
processing through RaaS. The percents are rounded to the nearest
tenth. {#tbl:error-causes}
'''

def replace_in_table(key, category, markdown, scripts_df, total):
    error_count = len(scripts_df[scripts_df["error_category"] == category].index)
    markdown = markdown.replace(key + "_COUNT", str(error_count))
    markdown = markdown.replace(key + "_PERCENT", "{0:.4g}".format(error_count / total * 100))
    return(markdown)

category_comparison_md = replace_in_table("LIBRARY", "library", category_comparison_md, scripts_df, num_error_scripts)
category_comparison_md = replace_in_table("WD", "working directory", category_comparison_md, scripts_df, num_error_scripts)
category_comparison_md = replace_in_table("FILE", "missing file", category_comparison_md, scripts_df, num_error_scripts)
category_comparison_md = replace_in_table("FUNC", "function", category_comparison_md, scripts_df, num_error_scripts)
category_comparison_md = replace_in_table("OTHER", "other", category_comparison_md, scripts_df, num_error_scripts)

category_comparison_md = category_comparison_md.replace("ERROR_TOTAL", str(num_error_scripts))

write_file_from_string("chen_category_comparison.md", category_comparison_md)

__Breakdown by Subject and Dataset__

This section requires the doi_metadata.json file generated by the ``get_doi_metadata.ipynb`` notebook.

We'll start by loading metadata scraped from Dataverse, and then converting it into a dataframe that will contain a row for each dataset. To indicate subjects, there will be a set of boolean columns, indicating whether the dataset was in that subject or not.

In [7]:
with open("../data/doi_metadata.json", "r") as doi_file:
    doi_metadata = json.loads(doi_file.read())

In [8]:
subject_set = set()
for doi_key in doi_metadata:
    if doi_metadata[doi_key][0] is not None:
        for subject in doi_metadata[doi_key][0]:
            subject_set.add(subject)            
subject_df = pd.DataFrame(columns = list(subject_set))
subject_df.insert(0, "year", [])
subject_df.insert(0, "doi", [])

In [21]:
df_dict = {"doi": [], "year":[]}
for subject in subject_set:
    df_dict[subject] = []

for doi_key in doi_metadata:
    if doi_metadata[doi_key][0] is not None:
        df_dict["year"].append(doi_metadata[doi_key][1])
        df_dict["doi"].append(doi_key.strip("\n"))
        for subject in subject_set:
            if subject in doi_metadata[doi_key][0]:
                df_dict[subject].append(True)
            else:
                df_dict[subject].append(False)

dataset_df = pd.DataFrame(df_dict)

In [35]:
def is_clean(doi, scripts_df):
    ret_val = None
    doi_df = scripts_df[scripts_df["doi"] == doi]
    if len(doi_df.index) > 0:
        ret_val = False
        errors = set(doi_df["error"].values)
        if "success" in errors and len(errors) == 1:
            ret_val = True
    return ret_val
is_clean_v = np.vectorize(is_clean, excluded=["scripts_df"])

In [41]:
clean_col = []
for doi in dataset_df["doi"].values:
    clean_col.append(is_clean(doi, scripts_df))
dataset_df["clean"] = clean_col

In [79]:
subject_breakdown_md = '''
-------------------------------------------------------------------------------------
  Subject                                Total Files   Total Error Files   Error Rate
  ------------------------------------ ------------- ------------------- ------------
  Social Sciences                              Social Sciences_TOTAL               Social Sciences_ERROR       Social Sciences_PERC%

  Computer and Information Science               Computer and Information Science_TOTAL                 Computer and Information Science_ERROR       Computer and Information Science_PERC%

  Medicine, Health and Life Sciences             Medicine, Health and Life Sciences_TOTAL                 Medicine, Health and Life Sciences_ERROR       Medicine, Health and Life Sciences_PERC%

  Physics                                         Physics_TOTAL                  Physics_ERROR       Physics_PERC%

  Engineering                                    Engineering_TOTAL                  Engineering_ERROR       Engineering_PERC%

  Other                                          Other_TOTAL                 Other_ERROR       Other_PERC%

  Business and Management                        Business and Management_TOTAL                 Business and Management_ERROR        Business and Management_PERC%

  Mathematical Sciences                          Mathematical Sciences_TOTAL                 Mathematical Sciences_ERROR       Mathematical Sciences_PERC%

  Arts and Humanities                            Arts and Humanities_TOTAL                 Arts and Humanities_ERROR       Arts and Humanities_PERC%

  Agricultural Sciences                          Agricultural Sciences_TOTAL                 Agricultural Sciences_ERROR       Agricultural Sciences_PERC%

  Law                                            Law_TOTAL                 Law_ERROR       Law_PERC%

  Earth and Environmental Sciences               Earth and Environmental Sciences_TOTAL                 Earth and Environmental Sciences_ERROR       Earth and Environmental Sciences_PERC%


-------------------------------------------------------------------------------------

  Table: This table contains the breakdown of error occurrences in R scripts
  on Dataverse by subject. Percentages are rounded to the nearest tenth. {#tbl:subject-breakdown}
'''

def replace_in_subject_table(key, markdown, subject_script_df):
    total = len(subject_script_df.index)
    error_count = len(subject_script_df[subject_script_df["error_category"] != "success"].index)
    markdown = markdown.replace(key + "_TOTAL", str(total))
    markdown = markdown.replace(key + "_ERROR", str(error_count))
    markdown = markdown.replace(key + "_PERC", "{0:.4g}".format(error_count / total * 100))
    return(markdown)

def get_subject_scripts(subject, dataset_df, scripts_df):
    dois_in_subject = dataset_df[dataset_df[subject] == True]["doi"].values
    scripts_from_doi = scripts_df[scripts_df.doi.isin(dois_in_subject)]
    return scripts_from_doi

subjects = []
for subject in subject_set:
    subject_script_df = get_subject_scripts(subject, dataset_df[["doi", subject]], scripts_df)
    subject_breakdown_md = replace_in_subject_table(subject, subject_breakdown_md, subject_script_df)

write_file_from_string("subject_breakdown.md", subject_breakdown_md)